## Concise Logistic Regression for Image Classification

- Shows a concise implementation of logistic regression for image classification
- Uses PyTorch

In [ ]:
# imports
import torch # 导包
import torchvision # 导包
import torch.nn as nn # 导包
from torchvision import datasets, models, transforms # 导包
import os # 导包
import numpy as np # 导包
import matplotlib.pyplot as plt # 导包

%matplotlib inline

# use gpu if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 有gpu就用，没有cpu

In [ ]:
# download the data (uncomment if to download the data locally)
#!wget https://download.pytorch.org/tutorial/hymenoptera_data.zip
#!unzip hymenoptera_data.zip

In [ ]:
# create data loaders

data_dir = 'data/hymenoptera_data' # 数据集path

# custom transformer to flatten the image tensors
class ReshapeTransform:
    def __init__(self, new_size):
        self.new_size = new_size # 初始化new_size

    def __call__(self, img):
        result = torch.reshape(img, self.new_size) # 重塑形状，new_size为-1就是转为1维
        return result # 返回

# transformations used to standardize and normalize the datasets
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        ReshapeTransform((-1,)) # flattens the data，new_size设为-1
    ]),
    'val': transforms.Compose([
        transforms.Resize(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        ReshapeTransform((-1,)) # flattens the data
    ]),
} # 

# load the correspoding folders
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']} # 加载数据集

# load the entire dataset; we are not using minibatches here
train_dataset = torch.utils.data.DataLoader(image_datasets['train'],
                                            batch_size=len(image_datasets['train']),
                                            shuffle=True) # 加载训练集

test_dataset = torch.utils.data.DataLoader(image_datasets['val'],
                                           batch_size=len(image_datasets['val']),
                                           shuffle=True) # 加载测试集

In [ ]:
# build the LR model
class LR(nn.Module): # 逻辑回归模型，继承nn.Module
    def __init__(self, dim): # 
        super(LR, self).__init__() # 父类初始化
        self.linear = nn.Linear(dim, 1) # 构建全连接层，y=wx+b
        nn.init.zeros_(self.linear.weight) # 初始化权重w
        nn.init.zeros_(self.linear.bias) # 初始化b

    def forward(self, x): # 前向传播，预测
        x = self.linear(x) # 线性回归结果
        x = torch.sigmoid(x) # 逻辑回归
        return x # 返回

In [ ]:
# predict function
def predict(yhat, y):
    yhat = yhat.squeeze() # 减少一维 torch.Size([244, 1]) -> torch.Size([244])
    y = y.unsqueeze(0) # 增加一维 torch.Size([244]) -> torch.Size([1, 244])
    y_prediction = torch.zeros(y.size()[1]) # 初始化预测值，shape:244
    for i in range(yhat.shape[0]):
        if yhat[i] <= 0.5: # 以0.5为界
            y_prediction[i] = 0 # 预测为0
        else:
            y_prediction[i] = 1 # 预测为1
    y_sub = y_prediction - y # 减法
    y_abs = torch.abs(y_sub) # 绝对值
    y_mean = torch.mean(y_abs) # 均值
    return 100 - y_mean * 100 # 返回得分，满分100

In [ ]:
# model config
dim = train_dataset.dataset[0][0].shape[0] # 维度

lrmodel = LR(dim).to(device) # 实例化模型
criterion = nn.BCELoss() # 目标值和预测值之间的二进制交叉熵损失函数
optimizer = torch.optim.SGD(lrmodel.parameters(), lr=0.0001) # 优化器使用SGD，学习率为0.0001

In [ ]:
# training the model
costs = [] # 代价值初始化

for ITER in range(100): # 循环100次
    lrmodel.train() # train模式
    x, y = next(iter(train_dataset)) # 取出一组
    test_x, test_y = next(iter(test_dataset)) # 取出一组

    # forward
    yhat = lrmodel.forward(x.to(device)) # 预测
    cost = criterion(yhat.squeeze(), y.type(torch.FloatTensor).to(device)) # 计算loss
    train_pred = predict(yhat, y) # 计算预测得分

    # backward
    optimizer.zero_grad() # 清空梯度
    cost.backward() # 梯度下降
    optimizer.step() # 参数更新
    
    # evaluate
    lrmodel.eval() # 设置为eval模式，相当于lrmodel.train(False)
    with torch.no_grad(): # torch.no_grad() 禁用梯度计算
        yhat_test = lrmodel.forward(test_x.to(device)) # 测试集预测值
        test_pred = predict(yhat_test, test_y) # 计算得分

    if ITER % 10 == 0:
        costs.append(cost) # 记录

    if ITER % 10 == 0:
        print("Cost after iteration {}: {} | Train Acc: {} | Test Acc: {}".format(ITER, 
                                                                                    cost, 
                                                                                    train_pred,
                                                                                    test_pred)) # 打印
   

### References
- [A Logistic Regression Model from Scratch](https://colab.research.google.com/drive/1iBoJ0kngkOthy7SgVaVQA1aHEROt5mra?usp=sharing)